# Example: NXWriter callback

**NOTE**: work-in-progress

**Objective**

Demonstrate use of `apstools.callbacks.NXWriter` and
`apstools.callbacks.NXWriterAPS` as callbacks and then use either (or both) to
export data from databroker.

**Contents**

- [Use as callback](#Use-as-callback)
- [Export data](#Export-data)

## Use as callback

```py
nxwriteraps = apstools.callbacks.NXWriterAPS()
RE.subscribe(nxwriteraps.receiver)
```

## Export data
